## Investigate Anomaly Detection Results



Check the parameters and search space for trends in anomaly detection.

The search space for the input parameters we're looking at yields an intractable computation.

The question being asked of the data is, *"what training period yields the greatest number of anomalies detected in summer-fall?"*.

Limiting the problem somewhat, the training period is set to to any combination of months in a single year.  The search space for this scenario is roughly 50K permutations.  The cost of search for a random subselection is roughly 1s per subset.  In other words, to run the anomaly detection algorithm (using the tensorflow backend which is already computationally optimized) on a random sample of 60 out of 50K options takes one minute. 141 stations takes over two hours.  While there may be room to optimize the code surrounding the central AD function, this segment of the research question focuses instead on optimization of the search itself, with the random search used as a benchmark.  At this point it should be stated that if all of the weather radar data were available, the anomaly detection step could be entirely avoided, to the benefit of the primary research question comparing radar precipitation and measured runoff.  However, the data are not available as such, and retrieving weather radar images is sufficiently costly to justify an event-based approach rather than seasons or years.

   

An efficient AD method should maximize the information gathered over a given number of weather radar data calls, commensurate with the number of viable rainfall-runoff events.  

The AD algorithm takes as input a matrix of streamflow observations, where each row represents an observation, and each column represents a marginal lag of the original daily flow series.  Principal Component Analysis is run on the normalized input dataset, and the number of components carried is determined by the number required to explain 90% of the variance.  In general, the number of components required to meet the 90% threshold is between 1 and 3.

The effectiveness of training data could simply be measured by the number of events detected, however several factors confound such metrics, namely the sheer number of events is too large for verification.  An information theoretic approach would measure the information content of the array of pixel values representing the catchment basin.  Consider the following bounding cases:  a sequence of radar images where no rainfall was detected by radar should yield a sequence of constant matrices of rgb values.  Such a scenario corresponds to low variance (low information).  On the other hand, a sequence of radar images capturing continuous dynamics of precipitation events would yield a sequence of matrices of rgb values with high variance (high information).  The efficacy of the training data selection algorithm may be better quantified by the variance in the data captured.  Since each basin has a unique shape, basins cannot be directly compared (in the same way the faces exercise was undertaken).  Instead, some unit measure could be used to compare information across watersheds, such as variance per pixel, though this may serve to add bias by encompassing the drainage area into the metric.


In [1]:
import os
import sys

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

from sklearn.decomposition import PCA

import colorcet as cc
from numpy import linspace
from scipy.stats.kde import gaussian_kde

from bokeh.io import output_file, show, output_notebook
from bokeh.models import ColumnDataSource, FixedTicker, PrintfTickFormatter
from bokeh.plotting import figure
from bokeh.sampledata.perceptions import probly
output_notebook()

Loading BokehJS ...

In [2]:
BASE_DIR = os.path.dirname(os.path.dirname(os.path.dirname(os.path.abspath(''))))
DB_DIR = os.path.join(BASE_DIR, 'code/hydat_db')
PROJECT_DIR = os.path.abspath('')
RESULTS_DIR = os.path.join(PROJECT_DIR, 'data/AD_results')


In [3]:
results_folders = os.listdir(RESULTS_DIR)

In [4]:
results_dict = {}
for folder in results_folders:
    folder_path = os.path.join(RESULTS_DIR, folder)
    n_samples = folder_path.split('_')[1].split('.')[0]
    all_sites = [e.split('_')[0] for e in os.listdir(folder_path)]
    results_dict[folder] = {site: pd.read_csv(os.path.join(folder_path, site + '_results.csv')) for site in all_sites}
    


In [5]:
print(results_dict.keys())

dict_keys(['sample_200'])


In [7]:
test_folder = 'sample_200'

In [17]:
results_df = results_dict[test_folder]

df = pd.concat([results_df[d] for d in results_df])
df.drop(labels='Unnamed: 0', axis=1, inplace=True)

In [18]:
# print(df.head())
# print(len(df))

In [19]:
months = []
len_results = []

months_dict = {m: [] for m in range(1,13,1)}

for idx, row in df.iterrows():
    months = [int(e) for e in row['train_months'].strip('(').strip(')').strip(' ').split(',') if e != '']
    
    for m in months:
        score = row['len_results']
        months_dict[m].append(score)
        
years = []
len_results = []
years_dict = {y: [] for y in range(2007, 2019, 1)}

max_score = 0
comp_results = []

for idx, row in df.iterrows():
    year = row['train_year']
    
    score = row['len_results']
    
    comp_results.append((row['num_components'], row['len_results']))
    
    years_dict[year].append(score)
    
    if score > max_score:
        max_score = score
       

In [38]:
comp_set = list(set([e[0] for e in comp_results]))
comp_dict = {c: [] for c in comp_set}
for e in comp_results:
    comp_dict[e[0]].append(e[1])
    
lb_min, ub_max, lb_2s, ub_2s, mean_vals, num_entries = [], [], [], [], [], []
for e in comp_dict:
    data = comp_dict[e]
    mean_vals.append(np.median(data))
    lb_2s.append(np.percentile(data, 95))
    ub_2s.append(np.percentile(data, 5))
    lb_min.append(np.percentile(data, 0.1))
    ub_max.append(np.percentile(data, 99.9))
    num_entries.append(len(data))

In [48]:
from bokeh.models import LinearAxis, Range1d

p = figure(plot_width=800, plot_height=400,
          x_range=(0, 11), y_range=(0, 30))

p.toolbar.logo = None
p.toolbar_location = None

p.varea(x=list(comp_dict.keys()), y1=lb_min, y2=ub_max, alpha=0.7,
           fill_color="#a9d6a9", legend_label="Max. Envelope (AD Performance)")

p.varea(x=list(comp_dict.keys()), y1=lb_2s, y2=ub_2s, alpha=0.8,
           fill_color="#53ac53", legend_label="95% conf. interval (AD Performance)")

p.line(list(comp_dict.keys()), mean_vals, line_width=2, 
       line_color='black', line_dash='dashed', legend_label="Median AD Performance")
p.yaxis.axis_label = 'AD Performance (# events identified)'
p.extra_y_ranges['n_components'] = Range1d(0, 15000)

ax2 = LinearAxis(y_range_name="n_components", 
                 axis_label="Sample Size for Min # PCA components > 90% variance")

# ax2.axis_label_text_color = "navy"
p.xaxis.axis_label = 'Number of PCA Components'
p.add_layout(ax2, 'right')
p.line(list(comp_dict.keys()), num_entries, line_width=2, legend_label="PCA Components",
       line_color='navy', line_dash='dotted', y_range_name="n_components")

# p.yaxis.axis_label = 'asdf'
show(p)

In [53]:
results_df50 = results_dict['sample_200']

df_50 = pd.concat([results_df50[d] for d in results_df50])
df_50.drop(labels='Unnamed: 0', axis=1, inplace=True)

# Monte Carlo simulation of selecting training period at random
sample_size = 1000
n_simulations = 1000
sim_df = pd.DataFrame()
sim_df['x'] = np.linspace(0, 50, 1000)

for i in range(n_simulations):
    random_sample = df_50.sample(n_simulations, replace=True)

    # fit a probability density function to the sample
    pdf = gaussian_kde(random_sample['len_results'])

    sim_df['sim_{}'.format(i)] = pdf(sim_df['x'])

    
sim_df['p_min'] = sim_df.quantile(q=0.01, axis=1)
sim_df['p_max'] = sim_df.quantile(q=0.99, axis=1)
sim_df['p_50'] = sim_df.quantile(q=0.5, axis=1)

In [54]:
p = figure(plot_width=800, plot_height=400, toolbar_location=None)

p.varea(x=sim_df['x'], y1=sim_df['p_min'], y2=sim_df['p_max'], alpha=0.7,
           fill_color="#a9d6a9", legend_label="95% conf. interval (AD Performance)")

p.line(sim_df['x'], sim_df['p_50'], line_color='black', legend_label='Median', 
       line_dash='dashed', line_width=2)

p.outline_line_color = None
p.background_fill_color = "#efefef"

# p.xaxis.ticker = FixedTicker(ticks=list(range(0, 101, 10)))
# p.xaxis.formatter = PrintfTickFormatter(format="%d%%")
p.xaxis.axis_label = 'Number of Events Detected'
p.yaxis.axis_label = 'P(X)'
p.ygrid.grid_line_color = None
p.xgrid.grid_line_color = "#dddddd"
# p.xgrid.ticker = p.xaxis.ticker

# p.axis.minor_tick_line_color = None
# p.axis.major_tick_line_color = None
p.axis.axis_line_color = None

# p.y_range.range_padding = 0.12

show(p)

In [ ]:
print(len(sim_df))
print(1E4)

In [ ]:
x = [y for y in years_dict]
y = [np.median(years_dict[a]) for a in years_dict]
y_err = [np.std(years_dict[a]) for a in years_dict]
fig, ax = plt.subplots(figsize=(8,6))
plt.errorbar(x, y, yerr=y_err, fmt='o')
print(x)

In [ ]:
# for each station, find the characteristics that 
# yield the highest return
test_folder = 'sample_50'

results_df5 = results_dict[test_folder]

df = pd.concat([results_df5[d] for d in results_df5])

months = []
len_results = []

months_dict = {m: [] for m in range(1,13,1)}

for idx, row in df.iterrows():
    months = [int(e) for e in row['train_months'].strip('(').strip(')').strip(' ').split(',') if e != '']
    

## Do PCA on the series of variables to see what explains the variability in the data.

Also, in the results dataframe, the radar station is showing the wsc station in error.

Fix in the Find_Events.py script.

In [ ]:
def do_PCA(X_train, X_test):
    
    for n_components in range(2, 6):

        pca = PCA(n_components=n_components, svd_solver= 'full')
        X_train_PCA = pca.fit_transform(X_train)
        X_train_PCA = pd.DataFrame(X_train_PCA)
        X_train_PCA.index = X_train.index

        X_test_PCA = pca.transform(X_test)
        X_test_PCA = pd.DataFrame(X_test_PCA)
        X_test_PCA.index = X_test.index

        var_expl = 100*np.sum(pca.explained_variance_ratio_)
        if var_expl >= 90:
            return X_train_PCA, X_test_PCA, var_expl, n_components
    return X_train_PCA, X_test_PCA, var_expl, n_components